# Securities Research Scanner Demo

This notebook demonstrates how to use the `SecurityScanner` class to filter and search for equities and ETFs based on various criteria.

The scanner allows you to:
- Filter by sector, industry, market cap, volatility, volume, and dividend yield
- Use local cached data or fetch live data from Yahoo Finance
- Return results as pandas DataFrame or JSON
- Combine multiple filters for complex queries

In [ ]:
# Import the SecurityScanner
from copilot_quant.research.scanner import SecurityScanner
import pandas as pd

# Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## Basic Usage

First, let's create a scanner instance. By default, it uses local data for fast filtering.

In [ ]:
# Initialize scanner with local data (fast, uses cached S&P 500 list)
scanner = SecurityScanner(data_source='local')
print(f"Scanner initialized with {len(scanner.df)} securities")

## Example 1: Find Specific Tickers

Let's start with a simple query - get information about specific stocks.

In [ ]:
# Find specific tickers with live data
results = scanner.find(
    tickers=['AAPL', 'MSFT', 'GOOGL', 'AMZN'],
    fetch_live_data=True  # Fetch current data from Yahoo Finance
)

print("\nTech Giants:")
print(results[['ticker', 'name', 'sector', 'market_cap', 'dividend_yield']])

## Example 2: Filter by Market Cap

Find large-cap technology stocks (market cap > $100B).

In [ ]:
# Find large-cap tech stocks with live data
large_cap_tech = scanner.find(
    sector='Technology',
    market_cap_min=100e9,  # $100 billion
    tickers=['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'ORCL', 'CSCO', 'ADBE', 'CRM'],
    fetch_live_data=True
)

print(f"\nFound {len(large_cap_tech)} large-cap tech stocks:")
print(large_cap_tech[['ticker', 'name', 'market_cap', 'avg_volume']].sort_values('market_cap', ascending=False))

## Example 3: Filter by Volatility

Find stocks with low volatility - good for conservative portfolios.

In [ ]:
# Find low-volatility defensive stocks
low_vol_stocks = scanner.find(
    volatility_max=0.20,  # Max 20% annualized volatility
    market_cap_min=50e9,  # $50B+
    tickers=['JNJ', 'PG', 'KO', 'PEP', 'WMT', 'COST', 'UNH'],
    fetch_live_data=True
)

print(f"\nFound {len(low_vol_stocks)} low-volatility stocks:")
print(low_vol_stocks[['ticker', 'name', 'volatility', 'dividend_yield']].sort_values('volatility'))

## Example 4: High Dividend Stocks

Find stocks with attractive dividend yields for income investors.

In [ ]:
# Find high-dividend stocks
dividend_stocks = scanner.find(
    dividend_yield_min=0.025,  # Minimum 2.5% yield
    market_cap_min=20e9,  # $20B+
    tickers=['JNJ', 'PFE', 'VZ', 'T', 'XOM', 'CVX', 'KO', 'PEP'],
    fetch_live_data=True
)

print(f"\nFound {len(dividend_stocks)} high-dividend stocks:")
print(dividend_stocks[['ticker', 'name', 'dividend_yield', 'sector']].sort_values('dividend_yield', ascending=False))

## Example 5: Complex Multi-Criteria Search

Combine multiple filters to find stocks matching specific investment criteria.

In [ ]:
# Find quality growth stocks:
# - Technology or Healthcare sector
# - Large cap ($200B+)
# - High liquidity (30M+ average volume)
# - Moderate volatility (< 30%)

quality_growth = scanner.find(
    market_cap_min=200e9,  # $200B+
    avg_volume_min=30e6,    # 30M+ shares/day
    volatility_max=0.30,    # < 30% volatility
    tickers=['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'JNJ', 'UNH', 'LLY'],
    fetch_live_data=True
)

print(f"\nFound {len(quality_growth)} quality growth stocks:")
print(quality_growth[['ticker', 'name', 'sector', 'market_cap', 'volatility', 'avg_volume']])

## Example 6: Exclude Specific Stocks

Filter results while excluding certain tickers.

In [ ]:
# Find tech stocks but exclude the "Magnificent 7"
other_tech = scanner.find(
    tickers=['ORCL', 'CSCO', 'ADBE', 'CRM', 'INTC', 'QCOM', 'TXN', 'AVGO'],
    exclude_tickers=['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA'],
    market_cap_min=50e9,
    fetch_live_data=True
)

print(f"\nFound {len(other_tech)} tech stocks (excluding Mag 7):")
print(other_tech[['ticker', 'name', 'market_cap']])

## Example 7: JSON Output Format

Get results in JSON format for API integration or export.

In [ ]:
# Get results as JSON (list of dictionaries)
json_results = scanner.find(
    tickers=['AAPL', 'MSFT'],
    fetch_live_data=True,
    as_json=True
)

print("\nJSON format output:")
import json
print(json.dumps(json_results, indent=2, default=str))

## Example 8: Using YFinance Data Source

For always-live data, you can initialize the scanner with yfinance mode.

In [ ]:
# Create scanner that always fetches live data
live_scanner = SecurityScanner(data_source='yfinance')

# Find current data (automatically fetches from Yahoo Finance)
live_results = live_scanner.find(
    tickers=['AAPL', 'MSFT', 'GOOGL'],
    market_cap_min=1e12  # $1 trillion+
)

print("\nLive data from yfinance:")
print(live_results[['ticker', 'name', 'market_cap', 'volatility']])

## Advanced Usage: Custom Screening Strategy

Build a complete screening strategy for value investing.

In [ ]:
# Value investing screen:
# - Established companies (market cap > $10B)
# - Good dividend yield (> 2%)
# - Lower volatility (< 25%)

value_stocks = scanner.find(
    market_cap_min=10e9,
    dividend_yield_min=0.02,
    volatility_max=0.25,
    tickers=['JNJ', 'PG', 'KO', 'PEP', 'XOM', 'CVX', 'JPM', 'BAC'],
    fetch_live_data=True
)

print(f"\nValue Investing Screen - Found {len(value_stocks)} stocks:")
print(value_stocks[['ticker', 'name', 'sector', 'market_cap', 'dividend_yield', 'volatility']])

# Calculate a simple score
if len(value_stocks) > 0:
    value_stocks['score'] = (
        value_stocks['dividend_yield'] * 100 -  # Higher dividend is better
        value_stocks['volatility'] * 100         # Lower volatility is better
    )
    print("\nRanked by Score (Div Yield - Volatility):")
    print(value_stocks[['ticker', 'name', 'dividend_yield', 'volatility', 'score']].sort_values('score', ascending=False))

## Error Handling

The scanner provides clear error messages when no securities match.

In [ ]:
# Try an impossible filter
try:
    impossible = scanner.find(
        market_cap_min=10e15,  # $10 quadrillion (doesn't exist!)
        tickers=['AAPL']
    )
except ValueError as e:
    print(f"\nExpected error: {e}")

## Summary

The `SecurityScanner` provides a flexible and powerful way to:

1. **Filter securities** by multiple criteria (sector, market cap, volatility, etc.)
2. **Use local or live data** depending on your needs
3. **Export results** in DataFrame or JSON format
4. **Build custom screens** for different investment strategies

### Available Filters:

- `sector`: Filter by GICS sector
- `industry`: Filter by industry
- `market_cap_min/max`: Market capitalization range
- `avg_volume_min`: Minimum average daily volume
- `volatility_min/max`: Volatility range (annualized)
- `dividend_yield_min/max`: Dividend yield range
- `asset_type`: 'equity' or 'etf'
- `tickers`: Include only specific tickers
- `exclude_tickers`: Exclude specific tickers

### Data Sources:

- `local`: Fast, uses cached S&P 500 list (default)
- `yfinance`: Always fetches live data from Yahoo Finance
- `fetch_live_data=True`: Fetch live data even with local scanner

### Next Steps:

- Integrate scanner results into backtesting strategies
- Build automated screening reports
- Create custom Streamlit UI for interactive screening
- Expand to include more asset classes (crypto, commodities, etc.)